<a href="https://colab.research.google.com/github/robin-chou/remember/blob/master/3D_Ken_Burns_multiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Ken Burns
This is a reference implementation of 3D Ken Burns Effect from a Single Image [1] using PyTorch. Given a single input image, it animates this still image with a virtual camera scan and zoom subject to motion parallax. Should you be making use of our work, please cite our paper [1].

<a href="https://arxiv.org/abs/1909.05483" rel="Paper"><img src="http://content.sniklaus.com/kenburns/paper.jpg" alt="Paper" width="100%"></a>


> Colab created by [Manuel Romero](https://twitter.com/mrm8488)

### 1. Clone the project

In [2]:
!git clone https://github.com/robin-chou/3d-ken-burns.git

Cloning into '3d-ken-burns'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 180 (delta 29), reused 36 (delta 14), pack-reused 125
Receiving objects: 100% (180/180), 349.42 KiB | 18.39 MiB/s, done.
Resolving deltas: 100% (102/102), done.


### 2. Download pretrained models

In [3]:
!wget -O ./3d-ken-burns/models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
!wget -O ./3d-ken-burns/models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
!wget -O ./3d-ken-burns/models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch

--2021-07-28 02:46:30--  http://content.sniklaus.com/kenburns/network-disparity.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 88.198.95.97
Connecting to content.sniklaus.com (content.sniklaus.com)|88.198.95.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 357438463 (341M) [application/octet-stream]
Saving to: ‘./3d-ken-burns/models/disparity-estimation.pytorch’

./3d-ken-burns/mode 100%[===================>] 340.88M  31.1MB/s    in 12s     

2021-07-28 02:46:42 (29.4 MB/s) - ‘./3d-ken-burns/models/disparity-estimation.pytorch’ saved [357438463/357438463]

--2021-07-28 02:46:42--  http://content.sniklaus.com/kenburns/network-refinement.pytorch
Resolving content.sniklaus.com (content.sniklaus.com)... 88.198.95.97
Connecting to content.sniklaus.com (content.sniklaus.com)|88.198.95.97|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2438928 (2.3M) [application/octet-stream]
Saving to: ‘./3d-ken-burns/models/disparity-ref

In [5]:
!curl https://colab.chainer.org/install | sh -
import chainer
chainer.print_runtime_info()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  22253      0 --:--:-- --:--:-- --:--:-- 22571
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9 MB 22 kB/s 
     |████████████████████████████████| 1.0 MB 54.9 MB/s 
+ set +ex
Installation succeeded!


/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda101) version 9.1.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda101>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


Platform: Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 7.8.0
ChainerX: Not Available
NumPy: 1.19.5
CuPy:
  OS                           : Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
  Python Version               : 3.7.11
  CuPy Version                 : 9.2.0
  CuPy Platform                : NVIDIA CUDA
  NumPy Version                : 1.19.5
  SciPy Version                : 1.4.1
  Cython Build Version         : 0.29.22
  Cython Runtime Version       : 0.29.23
  CUDA Root                    : /usr/local/cuda
  nvcc PATH                    : /usr/local/cuda/bin/nvcc
  CUDA Build Version           : 10000
  CUDA Driver Version          : 11020
  CUDA Runtime Version         : 10000
  cuBLAS Version               : 10000
  cuFFT Version                : 10000
  cuRAND Version               : 10000
  cuSOLVER Version             : (10, 0, 0)
  cuSPARSE Version             : 10000
  NVRTC Version                : (10, 0)
  Thrust Version               : 100903
  CUB Build Ve

In [6]:
import os
os.environ['CUDA_HOME'] = "/usr/local/cuda"

In [7]:
# !pip install moviepy


In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 5.6 MB 31.2 MB/s 
     |████████████████████████████████| 388 kB 53.3 MB/s 
     |████████████████████████████████| 251 kB 58.4 MB/s 
     |████████████████████████████████| 3.3 MB 55.0 MB/s 
     |████████████████████████████████| 26.9 MB 88 kB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110743 sha256=c80b743cf264368af8d88f41895b03912c580dc31758f6b895681b28f54f8458
  Stored in directory: /root/.cache/pip/wheels/56/dc/2b/9cd600d483c04af3353d66623056fc03faed76b7518faae4df
  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl size=6156 sha256=37750679c164cf7f6b5522c1564a133d4632815466da68013a13ee1fd913cf2d
  Stored in directory: /root/.cache/pip/wheels/12/36/1f/dc61e6ac10781d63cf6fa045eb09fa613a667384e12cb6e6e0
Successfully built moviepy proglog
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
  

In [8]:
cd /content/3d-ken-burns

/content/3d-ken-burns


In [9]:
!rm images/README.md
!mkdir results

### 3. Upload several a pictures from your filesystem

In [19]:
from google.colab import files

uploaded = files.upload()
for filename in uploaded.keys():
  !mv ./$filename ./images/$filename

Saving house1024.jpg to house1024.jpg
Saving NewHouse_1024.jpg to NewHouse_1024.jpg


### 4. Apply (bulk) 3D Ken Burns Effect

In [20]:
!for image in ./images/*; do python autozoom.py --in $image --out ./results/$(basename $image | cut -f1 -d '.').mp4; done

Moviepy - Building video ./results/house1024.mp4.
Moviepy - Writing video ./results/house1024.mp4

Moviepy - Done !
Moviepy - video ready ./results/house1024.mp4
Moviepy - Building video ./results/NewHouse_1024.mp4.
Moviepy - Writing video ./results/NewHouse_1024.mp4

Moviepy - Done !
Moviepy - video ready ./results/NewHouse_1024.mp4


#### Now you have all your videos under the folder **results** and you can *manually* right click and download them.

### If you want to *automatically* download all the videos run the following cell:

In [21]:
for filename in os.listdir("./results"):
  extension = os.path.splitext(filename)[-1].lower()
  if extension == ".mp4":
    files.download("./results/" + filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Did you enjoy? Buy me a coffe :)
 
[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)